## Quantile score

In [73]:
import numpy as np
import pandas as pd

from forecast_tools.baseline import Naive1

from typing import Union, Optional

In [123]:
def quantile_score(predicted_quantile: float,
                   actual: float,
                   prob: float) -> float:
    """
    Calculate Quantile Score for a single prediction interval.
    
    Parameters:
    predicted_quantile : float
        The predicted quantile value (e.g., 0.05, 0.5, etc.).
    actual : float
        The observed (actual) value.
    prob : float
        The probability associated with the predicted quantile (e.g., 0.05, 0.25, etc.).
    
    Returns:
    Quantile score for the given prediction interval.
    """
    if actual < predicted_quantile:
        loss = (1 - prob) * abs(predicted_quantile - actual)
    else:
        loss = prob * abs(predicted_quantile - actual)
    
    return 2 * loss


In [108]:
google_stock = pd.read_csv("./data/google_stock.csv", index_col="Date", parse_dates=True)
google_stock.tail()

,Unnamed: 0,Symbol,Open,High,Low,Close,Adj_Close,Volume,day
Date,,,,,,,,,
2018-12-24,1002,GOOG,973.900024,1003.539978,970.109985,976.219971,976.219971,1590300,1002
2018-12-26,1003,GOOG,989.010010,1040.000000,983.000000,1039.459961,1039.459961,2373300,1003
2018-12-27,1004,GOOG,1017.150024,1043.890015,997.000000,1043.880005,1043.880005,2109800,1004
2018-12-28,1005,GOOG,1049.619995,1055.560059,1033.099976,1037.079956,1037.079956,1414800,1005
2018-12-31,1006,GOOG,1050.959961,1052.699951,1023.590027,1035.609985,1035.609985,1493300,1006


In [109]:
train = google_stock[:"2016-01-04"]["Close"]
train.tail()

Date
2015-12-28    762.510010
2015-12-29    776.599976
2015-12-30    771.000000
2015-12-31    758.880005
2016-01-04    741.840027
Name: Close, dtype: float64

In [155]:
train = google_stock[:"2015-12-31"]["Close"].to_numpy()
test = google_stock["2016-01-04":]["Close"].to_numpy()[:5]
test

array([741.840027, 742.580017, 743.619995, 726.390015, 714.469971])

In [157]:
nf1 = Naive1()
nf1.fit(train)
pred, interval = nf1.predict(horizon=len(test), return_predict_int=True, alpha=[0.2])
pred, interval


(array([758.880005, 758.880005, 758.880005, 758.880005, 758.880005]),
 [array([[744.53997703, 773.22003297],
         [738.60014296, 779.15986704],
         [734.04234797, 783.71766203],
         [730.19994905, 787.56006095],
         [726.81472765, 790.94528235]])])

In [158]:
q1 = quantile_score(interval[0][0][0], test[0], prob=0.1)
q1

4.859910048669781

In [159]:
q2 = quantile_score(interval[0][0][1], test[0], prob=0.9)
q2

6.2760011945922525

In [160]:
round((q1 + q2) / 0.2, 2)

55.68

In [184]:
def quantile_scores(predicted_quantiles: float,
                    actuals: float,
                    prob: float) -> float:
    """
    Calculate Quantile Score for a single prediction interval.
    
    Parameters:
    predicted_quantile : float
        The predicted quantile value (e.g., 0.05, 0.5, etc.).
    actual : float
        The observed (actual) value.
    prob : float
        The probability associated with the predicted quantile (e.g., 0.05, 0.25, etc.).
    
    Returns:
    Quantile score for the given prediction interval.
    """

    if isinstance(predicted_quantiles, float):
        predicted_quantiles = np.array([predicted_quantiles])
        actuals = np.array([actuals])
    else:
        predicted_quantiles = np.asarray(predicted_quantiles)
        actuals = np.asarray(actuals)
        
    scores = [quantile_score(q, a, prob) for q, a, in zip(predicted_quantiles, actuals)]
    return np.array(scores)


In [185]:
interval[0][:,0]

array([744.53997703, 738.60014296, 734.04234797, 730.19994905,
       726.81472765])

In [192]:
q_lower = quantile_scores(interval[0][:,0], test, prob=0.1)
q_lower

array([ 4.85991005,  0.79597481,  1.91552941,  6.8578813 , 22.22056198])

In [193]:
q_upper = quantile_scores(interval[0][:,1], test, prob=0.9)
q_upper

array([ 6.27600119,  7.31597001,  8.01953341, 12.23400919, 15.29506227])

In [195]:
from forecast_tools.metrics import winkler_score

In [198]:
interval[0]

array([[744.53997703, 773.22003297],
       [738.60014296, 779.15986704],
       [734.04234797, 783.71766203],
       [730.19994905, 787.56006095],
       [726.81472765, 790.94528235]])

In [205]:
winkler_score(interval[0], test.reshape(-1, 1), alpha=0.2).round(2)

85.79